# Рекомендация тарифов

**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

**Цель проекта** - построить модель для задачи классификации, которая выберет подходящий тариф. Accuracy должен быть не менее 0.75.

**Описание имеющихся данных**



Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 
<br>
Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sklearn

from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn import tree
from tqdm import tqdm

## Шаг 1. Загрузка данных

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv', decimal=".")
except:
    df = pd.read_csv('users_behavior.csv', decimal=".")

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Загружены данные и выведена основная информация о датасете. Столбцы calls, minutes, messages, mb_used  являются признаками. Целевым признаком является стобец is_ultra. Также наблюдается мультиколлинеарность между звонками и минутами разговора. 

## Шаг 2. Разделите исходные данные на обучающую, валидационную и тестовую выборки.

In [7]:
# Извлекаем целевой признак из датафрейма
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Поскольку у нас нет отложенной тестовой выборки, то придется из текущего датасета выделять обучающую, валидационную и тестовую выборки в соотношении 3:1:1 (60% : 20% : 20%)

In [8]:
# сначала необходимо выделить обучающую выборку из всего датасета
features_train, features_check, target_train, target_check = train_test_split(features, target, train_size=0.6, random_state=12345, stratify=target)
# затем выделяем тестовую и валдиационную выборки 
features_test, features_valid, target_test, target_valid = train_test_split(features_check, target_check, test_size=0.5, random_state=12345, stratify=target_check)

print(np.bincount(target_train))
print(np.bincount(target_valid))
print(np.bincount(target_test))

[1337  591]
[446 197]
[446 197]


In [9]:
print(len(features_train['calls']))
print(len(features_valid['calls']))
print(len(features_test['calls']))


1928
643
643


## Шаг 3. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

### Модель 1. Decision Tree

In [10]:
%%time
best_result = 0
for split in range(2, 10):
    for leaf in range(1, 10):
        for depth in range(1, 11):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_leaf=leaf, min_samples_split=split)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            accuracy = accuracy_score(target_valid, predictions)
            if accuracy > best_result:
                best_result = accuracy
                best_depth = depth
                best_leaf = leaf
                best_split = split
print("Наивысшее значение accuracy = ", best_result)
print("Наивысшее значение accuracy при depth = ", best_depth)
print("Наивысшее значение accuracy при min_samples_leaf = ", best_leaf)
print("Наивысшее значение accuracy при min_samples_split = ", best_split)

Наивысшее значение accuracy =  0.8118195956454122
Наивысшее значение accuracy при depth =  5
Наивысшее значение accuracy при min_samples_leaf =  1
Наивысшее значение accuracy при min_samples_split =  2
CPU times: user 4.12 s, sys: 14.2 ms, total: 4.13 s
Wall time: 4.14 s


Для модели Decision Tree подобрали гиперпараметры max_depth = 5, min_samples_split = 2, min_samples_leaf = 1 для достижения максимального значения Accuracy.

In [ ]:
clf = tree.DecisionTreeClassifier(random_state=12345, max_depth=3, min_samples_leaf=1, min_samples_split=2)
clf = clf.fit(features_train, target_train)
plt.figure(figsize=(25,20))
tree.plot_tree(clf);

### Модель 2. Random Forest

In [12]:
best_result = 0
for est in tqdm(range(1, 100, 10)):
    for depth in range(1, 20):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions) 
        if result > best_result:
            best_result = result
            best_depth = depth
            best_est = est
print("Наивысшее значение accuracy = ", best_result)
print("Наивысшее значение accuracy при depth = ", best_depth)
print("Наивысшее значение accuracy при n_est = ", best_est)

100%|██████████| 10/10 [00:25<00:00,  2.59s/it]

Наивысшее значение accuracy =  0.8242612752721618
Наивысшее значение accuracy при depth =  12
Наивысшее значение accuracy при n_est =  31


Для модели Random Forest подобрали гиперпараметры n_estimators=31 и max_depth=12 для достижения максимального значения Accuracy.

### Модель 3. Logistic Regression

**Гиперпараметр solver = lbfgs**

In [13]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
accuracy

0.7465007776049767

Для модели Logistic Regression оценено значение accuracy.

**Вывод:** В ходе исследования 3-х различных моделей наивысшего значения accuracy для валидационной выборки удалось добиться в модели Random Forest равного почти 0.82, что является весьма хорошим показателем. На втором месте расположилась модель Decision Tree, а замыкает тройку логистическая регрессия. Подобраны оптимальные гиперпараметры для максимизации accuracy.

## Шаг 4. Проверьте качество модели на тестовой выборке.

### Обучаем модель на обучающей выборке

In [14]:
model = RandomForestClassifier(random_state=12345, n_estimators=31, max_depth=12)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
accuracy

0.7978227060653188

**Вывод:** При проверке качества модели RF на тестовой выборке получено значение accuracy = 0.797

### Обучаем модель на соединенной выборке из обучающей и валидационной

In [15]:
features_new = pd.concat([features_train, features_valid])
target_new = pd.concat([target_train, target_valid])
print(features_new.shape)
print(target_new.shape)

(2571, 4)
(2571,)


In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=31, max_depth=12)
model.fit(features_new, target_new)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
accuracy

0.8149300155520995

**Вывод:** При увеличении обучающей выборки получилось увеличить метрику accuracy для тестовой выборки.

## Шаг 5. Дополнительное задание: проверьте модели на вменяемость.

Для проверки адекватности модели машинного обучения в Python, удобно использовать DummyClassifier из библиотеки scikit-learn. Это простой классификатор, который можно использовать в качестве эталонной модели.
<br>
После обучения этой модели можно провести тестирование и результаты использовать для сравнения с результатами более сложных моделей. Например, если результаты DummyClassifier близки к результатам другой модели, то можно предположить, что более сложная модель не имеет смысла использовать в данной задаче.
<br>

Важно учитывать, что DummyClassifier не учитывает никаких особенностей данных и использует случайные значения для классификации, поэтому его результаты не могут быть использованы для принятия окончательных решений, а лишь для проверки адекватности моделей.

In [17]:
classifier_dummy = DummyClassifier(strategy='most_frequent', random_state=12345)
classifier_dummy.fit(features_train, target_train)
predictions = classifier_dummy.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
accuracy

0.6936236391912908

**Вывод:** Модели прошли проверку на вменяемость, поскольку их значение accuracy больше, чем у случайной модели.

**Общий вывод:** В ходе проделанного анализа выполнены следующие мероприятия:
- Загружены и проанализированы данные о поведении пользователей оператора мобильной связи "Мегалайн".
- Разделены исходные данные на обучающую, валидационную и тестовую выборки
- Исследовано качество 3-х моделей, где подобраны оптимальные гиперпараметры для максимизации accuracy. Наибольшего значения для валидационной выборки удалось добиться для модели Decion Tree.
- При проверке качества моделей на тестовой выборке наилучшей (с небольшим перевесом) оказалась модель Random Forest.
- Все 3 модели оказались вменяемыми.